Universidad del Valle de Guatemala
Security Data Science
Sección 10

# Laboratorio 1
## Detección de Pishing

Walter Saldaña #19897
___

In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

In [2]:
%pip install pandas-profiling
import pandas_profiling

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_7763/2731892375.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


# Parte 1
## Exploración de datos
1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [3]:
df = pd.read_csv('dataset_pishing.csv')
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset? 

In [4]:
df[df.status == 'legitimate'].status.count()

5715

In [5]:
df[df.status == 'phishing'].status.count()

5715

El dataset está balanceado con exactamente en partes iguales para observaciones de legitimate y phishing.

## Derivación de características 
En  base  al  artículo  “Towards  Benchmark  Datasets  for  ML  Based  Wensite  Phishing  Detection:  An 
Experimental Study”, derivar las características basadas en el dominio: f1, f2, f4 – f20, f25, f26 y f27. 
 
Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 

### f1
Full URL lenght

In [6]:
df['url_length'] = df['url'].str.len()
df.url_length

0         37
1         77
2        126
3         18
4         55
        ... 
11425     45
11426     84
11427    105
11428     38
11429    477
Name: url_length, Length: 11430, dtype: int64

### f2
Hostname lenght

In [7]:
def get_hostname(url):
    new_url = url.replace('https://', '')
    new_url = new_url.replace('http://', '')
    new_url = new_url.split('/')[0]
    return new_url

df['hostname_length'] = df['url'].apply(get_hostname).str.len()
df.hostname_length

0        19
1        23
2        50
3        11
4        15
         ..
11425    17
11426    18
11427    16
11428    30
11429    14
Name: hostname_length, Length: 11430, dtype: int64

### f4-20
Special characters
'.' (f4), ’-’ (f5), ’@’ (f6) , ’?’ (f7), ’&’ (f8), ’|’ (f9), ’=’ (f10), ’_’ (f11), ’ ̃’ (f12),
’%’ (f13), ’/’ (f14), ’*’ (f15), ’:’ (f16), ’,’ (f17), ’;’ (f18), ’$’ (f19), ’%20’ or
space (f20).

In [8]:
chars = ['.', '-', '@', '?', '&', '|', '=', '_', " ̃", '%', '/', '*', ':', ',', ';', '$', '%20', ' ']
c = 0

def count_chars(value):
    return sum(
        i == chars[c]
        for i
        in value
    )

for i in range(4, 21):
    col_name = f'f{i}'
    df[col_name] = df['url'].apply(count_chars)
    c += 1
df['f20'] = df['f20'] + df['url'].apply(count_chars)
    
df

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,0,0,3,0,1,0,0,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,0,0,0,5,0,1,0,0,0,0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,2,0,0,5,0,1,0,0,0,0
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,0,0,5,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,0,0,0,0,...,0,0,0,4,0,1,0,0,0,0
11426,http://www.budgetbots.com/server.php/Server%20...,0,1,1,0,0,0,0,1,0,...,0,0,1,5,0,1,0,0,0,0
11427,https://www.facebook.com/Interactive-Televisio...,1,1,0,0,0,0,0,0,0,...,1,0,0,5,0,1,0,0,0,0
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,0,0,0,0,0,...,0,0,0,3,0,1,0,0,0,0


### f25
Https certificate

In [9]:
df['f25'] = df['url'].str.contains('https://').astype(int)
df

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f12,f13,f14,f15,f16,f17,f18,f19,f20,f25
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,0,3,0,1,0,0,0,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,0,0,5,0,1,0,0,0,0,0
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,0,0,5,0,1,0,0,0,0,1
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,0,0,2,0,1,0,0,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,0,5,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,0,0,0,0,...,0,0,4,0,1,0,0,0,0,0
11426,http://www.budgetbots.com/server.php/Server%20...,0,1,1,0,0,0,0,1,0,...,0,1,5,0,1,0,0,0,0,0
11427,https://www.facebook.com/Interactive-Televisio...,1,1,0,0,0,0,0,0,0,...,0,0,5,0,1,0,0,0,0,1
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,0,0,0,0,0,...,0,0,3,0,1,0,0,0,0,0


### f26
Digits ratio in url

In [10]:
def count_digits(string):
    return sum(
        item.isdigit()
        for item
        in string
    )

df['f26'] = df['url'].apply(count_digits) / df['url_length']
df

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f13,f14,f15,f16,f17,f18,f19,f20,f25,f26
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,3,0,1,0,0,0,0,0,0.000000
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,0,5,0,1,0,0,0,0,0,0.220779
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,0,5,0,1,0,0,0,0,1,0.150794
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,0.000000
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,5,0,1,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,0,0,0,0,...,0,4,0,1,0,0,0,0,0,0.000000
11426,http://www.budgetbots.com/server.php/Server%20...,0,1,1,0,0,0,0,1,0,...,1,5,0,1,0,0,0,0,0,0.023810
11427,https://www.facebook.com/Interactive-Televisio...,1,1,0,0,0,0,0,0,0,...,0,5,0,1,0,0,0,0,1,0.142857
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,0,0,0,0,0,...,0,3,0,1,0,0,0,0,0,0.000000


### f27
Digits ratio in hostname

In [11]:
def count_digits_hn(string):
    string = get_hostname(string)
    return sum(
        item.isdigit()
        for item
        in string
    )

df['f27'] = df['url'].apply(count_digits_hn) / df['hostname_length']
df

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.220779,0.000000
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,5,0,1,0,0,0,0,1,0.150794,0.000000
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,0.000000,0.000000
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,0,0,0,0,...,4,0,1,0,0,0,0,0,0.000000,0.000000
11426,http://www.budgetbots.com/server.php/Server%20...,0,1,1,0,0,0,0,1,0,...,5,0,1,0,0,0,0,0,0.023810,0.000000
11427,https://www.facebook.com/Interactive-Televisio...,1,1,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,1,0.142857,0.000000
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000


## Preprocesamiento 
Realice  las  modificaciones  necesarias  para  convertir  la  variable  categórica  status  a  una  variable 
binaria. Elimine la columna del dominio. 

In [12]:
df['status'] = np.where(df['status'] == 'phishing', 1, 0)
df.status

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: int64

In [13]:
df = df.drop(['url'], axis=1)
df

,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,0,1,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000
1,1,0,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.220779,0.000000
2,1,0,1,0,0,0,0,0,1,0,...,5,0,1,0,0,0,0,1,0.150794,0.000000
3,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,0.000000,0.000000
4,0,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,0,1,0,0,0,0,0,0,0,0,...,4,0,1,0,0,0,0,0,0.000000,0.000000
11426,0,1,1,0,0,0,0,1,0,0,...,5,0,1,0,0,0,0,0,0.023810,0.000000
11427,1,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,1,0.142857,0.000000
11428,0,1,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000


## Visualización de resultados 
Genere  un  reporte  de  perfil  con  la  librería  pandas_profiling.  Analice  el  reporte  y  determine  las 
columnas que son constantes, o que no tienen una varianza alta con la columna status. Almacene su 
reporte como una página html. 

In [14]:
profile = pandas_profiling.ProfileReport(df)
#profile.to_file("profile.html")

## Selección de Características 
En base al análisis del reporte, elimine las características repetidas o irrelevantes para la clasificación 
de un sitio de pishing. Verifique que no posee observaciones repetidas. 

In [15]:
corr = df.corr()
corr.style.background_gradient()

/home/walt/.local/lib/python3.9/site-packages/pandas/io/formats/style.py:3555: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/home/walt/.local/lib/python3.9/site-packages/pandas/io/formats/style.py:3556: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


In [16]:
df = df.drop(['avg_word_host'], axis=1)
df = df.drop(['f10'], axis=1)
df

,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,...,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
0,0,1,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000
1,1,0,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.220779,0.000000
2,1,0,1,0,0,0,0,0,1,0,...,5,0,1,0,0,0,0,1,0.150794,0.000000
3,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,0.000000,0.000000
4,0,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,0,1,0,0,0,0,0,0,0,0,...,4,0,1,0,0,0,0,0,0.000000,0.000000
11426,0,1,1,0,0,0,0,1,0,0,...,5,0,1,0,0,0,0,0,0.023810,0.000000
11427,1,1,0,0,0,0,0,0,0,0,...,5,0,1,0,0,0,0,1,0.142857,0.000000
11428,0,1,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0.000000,0.000000


# Parte 2 – Implementación 
## Separación de datos 
- Datos de entrenamiento: 55% 
- Datos de validación: 15% 
- Datos de prueba: 30% 
- Almacene cada dataset como un archivo .csv 

In [17]:
target = df['status']
feature_matrix = df.drop(['status'], axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(feature_matrix, target, test_size=0.3, random_state=33)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.15, random_state=33)

In [18]:
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y val shape: {}".format(y_val.shape))

X_train shape: (6800, 85)
X_test shape: (3429, 85)
y_train shape: (6800,)
y_test shape: (3429,)
X_val shape: (1201, 85)
y val shape: (1201,)


## Implementación 
Utilice dos algoritmos de Machine Learning para entrenar el modelo. Muestre y explique los valores 
obtenidos de las siguientes métricas para los datos de validación y pruebas, para cada modelo, en 
base al contexto del problema (detección de Pishing). 
- Matriz de confusión 
- Precision
- Recall 
- F1 Score

### Modelo 1: Árbol de decisión

In [19]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

test_feature = X_test[192:193]
test_target = y_test[192:193]

pred = clf.predict(test_feature)
print('Predicted class:', pred)
print('Accurate prediction?', pred[0] == test_target)

Predicted class: [0]
Accurate prediction? 6136    True
Name: status, dtype: bool


In [20]:
pred[0] == test_target

6136    True
Name: status, dtype: bool

In [21]:
y_pred = clf.predict(X_test)

#### Confusion matrix

In [22]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(1547, 115, 119, 1648)

#### Precision

In [23]:
precision = tp / (tp + fp)
precision

0.9347702779353375

#### Recall

In [24]:
recall = tp / (tp + fn)
recall

0.9326542161856254

#### F1 Score

In [25]:
2 * (precision * recall) / (precision + recall)

0.9337110481586403

### Modelo 2: Gaussian Naive Bayes

In [26]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

#### Confusion matrix

In [27]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(1062, 600, 253, 1514)

#### Precision

In [28]:
precision = tp / (tp + fp)
precision

0.7161778618732261

#### Recall

In [29]:
recall = tp / (tp + fn)
recall

0.8568194680249009

#### F1 Score

In [30]:
2 * (precision * recall) / (precision + recall)

0.7802112857510951

# Discusión 
1. **¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?**
Puede implicar pérdida de información, cortar la comunicación, pues desestimaría los correos que se detecten como phishing que realmente fueron enviados por sitios legitimos, probablemente enviandolos a spam y que por lo tanto el destinatario no lo vea.

2. **¿Cuál es el impacto de clasificar un sitio de Pishing como legítimo?**
Si el usuario no es precavido, puede confiar en el mensaje de phishing y caer en la estafa del mismo, ya sea dando información valiosa que se pueda usar en contra de la organización, perdiendo dinero, clientes, etc.

3. **En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares de clasificación de pishing?**
Importa más proteger al usuario y evitar riesgos, por lo que con este fin es relativamente aceptable que un correo importante termine en spam al compararlo con el impacto de ceder información interna a externos por medio de phishing, por lo tanto, la métrica que elegiría para comparar los modelos de clasificación es el RECALL, pues esta métrica toma en consideración los false negative, es decir, los correos de phishing que se etiquetaron como legítimos.

4. **¿Qué modelo es mejor para la clasificación de Pishing? Justifique**
El modelo que dió mejor desempeño para clasificar el Phishing es el de árboles de decisión, pues obtuvo un recall de 93%, comparado con el modelo de redes bayesianas que obtuvo un 85%.

5. **En base a las métricas obtenidas ¿es necesaria la intervención de una persona humana para tomar la decisión final?** 
Si, es mejor alertar sobre un posible phishing en lugar de censurar la información por completo sin la decisión del usuario, pues de esta forma mitigamos el riesgo de caer al phishing al advertirle al usuario que cierto correo es problable que sea phishing, pero al mismo tiempo evitando perder información valiosa que pudo no haber sido phishing.